In [1]:
import os

# Función para instalar OpenJDK 8 si no está instalado previamente.
def install_java():
    # Verifica si la ruta del JDK existe.
    if not os.path.exists('/usr/lib/jvm/java-8-openjdk-amd64'):
        print("Instalando OpenJDK 8...")
        # Comando para instalar Java en modo silencioso.
        !apt-get install openjdk-8-jdk-headless -qq > /dev/null
        print("OpenJDK 8 instalado correctamente.")
    else:
        print("OpenJDK 8 ya está instalado.")



# Función para descargar Apache Spark solo si no está ya descargado.
def download_spark():
    # URL y nombre del archivo comprimido de Spark.
    spark_url = "https://archive.apache.org/dist/spark/spark-3.4.3/spark-3.4.3-bin-hadoop3.tgz"
    spark_tar = "spark-3.4.3-bin-hadoop3.tgz"

    # Verifica si el archivo comprimido ya existe.
    if not os.path.exists(spark_tar):
        print("Descargando Apache Spark...")
        # Comando para descargar el archivo desde la URL.
        !wget -q $spark_url
        print("Descarga completa.")
    else:
        print("El archivo de Apache Spark ya está descargado.")



# Función para descomprimir el archivo de Apache Spark solo si no está descomprimido.
def extract_spark():
    # Nombre de la carpeta descomprimida.
    spark_dir = "spark-3.4.3-bin-hadoop3"
    spark_tar = "spark-3.4.3-bin-hadoop3.tgz"

    # Verifica si la carpeta descomprimida ya existe.
    if not os.path.exists(spark_dir):
        print("Descomprimiendo Apache Spark...")
        # Comando para descomprimir el archivo.
        !tar xf $spark_tar
        print("Apache Spark descomprimido.")
    else:
        print("La carpeta de Apache Spark ya existe.")



# Función para configurar las variables de entorno necesarias para Spark.
def set_environment_variables():
    # Establece la ruta de JAVA_HOME y SPARK_HOME.
    os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
    os.environ["SPARK_HOME"] = "/content/spark-3.4.3-bin-hadoop3"
    print("Variables de entorno configuradas.")



# Función para verificar si un paquete de Python ya está instalado.
import subprocess
import sys

def is_package_installed(package_name):
    try:
        # Ejecuta el comando `pip show` para verificar si el paquete está instalado.
        subprocess.check_call(
            [sys.executable, "-m", "pip", "show", package_name],
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL
        )
        return True
    except subprocess.CalledProcessError:
        return False



# Función para instalar librerías de Python necesarias (findspark y pyspark).
def install_python_libraries():
    # Verifica e instala findspark.
    if not is_package_installed("findspark"):
        print("Instalando findspark...")
        !pip install -q findspark
    else:
        print("findspark ya está instalado.")

    # Verifica e instala pyspark.
    if not is_package_installed("pyspark"):
        print("Instalando pyspark...")
        !pip install -q pyspark
    else:
        print("pyspark ya está instalado.")



install_java()                # Instala OpenJDK 8
download_spark()              # Descarga Apache Spark
extract_spark()               # Descomprime Apache Spark
set_environment_variables()   # Configura las variables de entorno
install_python_libraries()    # Instala las librerías de Python

OpenJDK 8 ya está instalado.
El archivo de Apache Spark ya está descargado.
La carpeta de Apache Spark ya existe.
Variables de entorno configuradas.
findspark ya está instalado.
pyspark ya está instalado.


---

# **Crear una sesion en Spark**

In [2]:
# Importar la biblioteca findspark, que ayuda a configurar PySpark correctamente en el entorno de ejecución
import findspark

# Inicializar findspark para establecer las variables necesarias de PySpark
findspark.init()

# Importar SparkSession desde la biblioteca pyspark.sql
# SparkSession es la entrada principal para trabajar con DataFrames en PySpark
from pyspark.sql import SparkSession

# Crear o obtener una SparkSession
# Esto inicializa un entorno de Spark si no existe uno, o reutiliza uno existente
spark = SparkSession.builder.getOrCreate()

# Obtener el contexto de Spark (SparkContext) desde la SparkSession
# SparkContext es la entrada principal para trabajar con RDDs en PySpark
sc = spark.sparkContext


---

# Funcion `reduce`

In [4]:
# Crear un RDD (Resilient Distributed Dataset) con los números 1, 2, 3, 4, 5
rdd = sc.parallelize([1, 2, 3, 4, 5])

# Usar la función reduce para sumar todos los elementos del RDD
# La función lambda toma dos argumentos (x, y) y devuelve su suma
resultado = rdd.reduce(lambda x, y: x + y)

# Imprimir el resultado de la suma
print(resultado)

15


# Funcion `map`

In [5]:
# Crear un RDD con los números 1, 2, 3, 4, 5
rdd = sc.parallelize([1, 2, 3, 4, 5])

# Transformar cada número del RDD: multiplicarlo por 2 y luego elevarlo al cuadrado
rdd_transformado = rdd.map(lambda x: (x * 2) ** 2)

# Recoger los resultados transformados en una lista para visualización
resultado_transformado = rdd_transformado.collect()

# Imprimir los números transformados
print(resultado_transformado)

[4, 16, 36, 64, 100]


La diferencia principal entre `reduce` y `map` radica en **qué operación realizan y su propósito**:

1. **`reduce`**:  
   - Se usa para **reducir** todos los elementos de un RDD a un único valor combinándolos según una función.  
   - En el ejemplo original, `reduce(lambda x, y: x + y)` toma dos números del RDD, los suma, y repite este proceso hasta que queda un solo valor, que es la suma total.  

   ```python
   resultado = rdd.reduce(lambda x, y: x + y)
   # Suma todos los elementos del RDD.
   ```

2. **`map`**:  
   - Se usa para **transformar** cada elemento del RDD aplicándole una función y devolviendo un nuevo RDD con los resultados.  
   - En el código nuevo, `map(lambda x: (x * 2) ** 2)` toma cada número del RDD, lo multiplica por 2, lo eleva al cuadrado y genera un nuevo RDD con los valores transformados.  

   ```python
   rdd_transformado = rdd.map(lambda x: (x * 2) ** 2)
   # Transforma cada elemento del RDD.
   ```

### Resumen:  
- **`reduce`**: Reduce todo el conjunto a un único valor.  
- **`map`**: Aplica una transformación a cada elemento, generando un nuevo conjunto con los valores transformados.  

---

# Funcion `count`

La función **`count`** en PySpark se usa para contar el número total de elementos en un RDD o DataFrame. Es útil para obtener el tamaño del conjunto de datos con el que estás trabajando.

In [7]:
# Crear un RDD con letras
rdd_letras = sc.parallelize(["a", "b", "c", "a", "b", "a"])

# Contar cuántos elementos hay en el RDD
conteo_letras = rdd_letras.count()

# Imprimir el resultado
print(f"El número total de letras es: {conteo_letras}")

El número total de letras es: 6


In [9]:
# Crear un RDD con números utilizando range
rdd_numeros = sc.parallelize(range(1, 11))

# Contar cuántos elementos hay en el RDD
conteo_numeros = rdd_numeros.count()

# Imprimir el resultado
print(f"El número total de números es: {conteo_numeros}")

El número total de números es: 10


In [10]:
# Crear un RDD con valores booleanos
rdd_booleanos = sc.parallelize([True, False, True, False, True])

# Contar cuántos elementos hay en el RDD
conteo_booleanos = rdd_booleanos.count()

# Imprimir el resultado
print(f"El número total de valores booleanos es: {conteo_booleanos}")

El número total de valores booleanos es: 5


---

# Funcion `collect`

La función **`collect`** en PySpark se utiliza para recuperar todos los elementos de un RDD o DataFrame y traerlos al programa principal (driver) como una lista. Es útil para inspeccionar datos pequeños, pero debe usarse con cuidado en conjuntos de datos grandes para evitar problemas de memoria.

In [11]:
# Crear un RDD con números
rdd_numeros = sc.parallelize(range(1, 6))

# Usar collect para traer los datos al programa principal
resultado_numeros = rdd_numeros.collect()

# Imprimir los números recogidos
print(f"Los números en el RDD son: {resultado_numeros}")

Los números en el RDD son: [1, 2, 3, 4, 5]


In [12]:
# Crear un RDD con cadenas de texto
rdd_texto = sc.parallelize(["hola", "mundo", "apache", "spark"])

# Usar collect para traer los datos al programa principal
resultado_texto = rdd_texto.collect()

# Imprimir las cadenas recogidas
print(f"Las cadenas en el RDD son: {resultado_texto}")

Las cadenas en el RDD son: ['hola', 'mundo', 'apache', 'spark']


In [13]:
# Crear un RDD con valores booleanos
rdd_booleanos = sc.parallelize([True, False, True, True, False])

# Usar collect para traer los datos al programa principal
resultado_booleanos = rdd_booleanos.collect()

# Imprimir los valores booleanos recogidos
print(f"Los valores booleanos en el RDD son: {resultado_booleanos}")

Los valores booleanos en el RDD son: [True, False, True, True, False]


In [18]:
# Filtrar números pares y multiplicarlos por 3
# Crear un RDD con números del 1 al 20
rdd_numeros = sc.parallelize(range(1, 21))

# Filtrar los números pares
rdd_pares = rdd_numeros.filter(lambda x: x % 2 == 0)

# Multiplicar los números pares por 3
rdd_transformado = rdd_pares.map(lambda x: x * 3)

# Usar collect para traer los resultados transformados al programa principal
resultado = rdd_transformado.collect()

# Imprimir los resultados
print(f"Los números pares multiplicados por 3 son: {resultado}")

Los números pares multiplicados por 3 son: [6, 12, 18, 24, 30, 36, 42, 48, 54, 60]


In [25]:
# Filtrar numero pares, elevarlos al cuadrado e imprimir tuplas (numero, numero**2)

# Crear un RDD con números del 1 al 20
rdd_numeros = sc.parallelize(range(1, 21))

# Filtrar los números pares
rdd_pares = rdd_numeros.filter(lambda x: x % 2 == 0)

# Crear tuplas (número, número**2)
rdd_tuplas = rdd_pares.map(lambda x: (x, x ** 2))

# Usar collect para traer los resultados transformados al programa principal
resultado = rdd_tuplas.collect()

# Imprimir los resultados
print(f"Las tuplas (número, número**2) son: {resultado}")


Las tuplas (número, número**2) son: [(2, 4), (4, 16), (6, 36), (8, 64), (10, 100), (12, 144), (14, 196), (16, 256), (18, 324), (20, 400)]


---

**Funciones take, max, y saveAsTextFile en PySpark**

### Introducción
En PySpark, existen funciones que permiten realizar operaciones específicas en los RDDs para obtener resultados o guardar datos. A continuación, se explican las funciones **`take`**, **`max`**, y **`saveAsTextFile`**, junto con ejemplos de uso y sus aplicaciones.

---

### 1. Función `take`

#### Descripción:
La función **`take`** se utiliza para recuperar un número específico de elementos del RDD, empezando desde el inicio. Es especialmente útil para inspeccionar los primeros elementos de un conjunto de datos sin necesidad de procesar todo el RDD.

#### Sintaxis:
```python
rdd.take(n)
```
- **`n`**: El número de elementos que se desea recuperar.

#### Ejemplo de uso:

In [27]:
# Crear un RDD con números del 1 al 20
rdd = sc.parallelize(range(1, 21))

# Tomar los primeros 5 elementos
resultado = rdd.take(5)

# Imprimir el resultado
print(f"Los primeros 5 elementos son: {resultado}")

Los primeros 5 elementos son: [1, 2, 3, 4, 5]


In [30]:
rdd = sc.parallelize('La programacion y la ciencia de datos son increibles. Me gusta todo esto!!'.split(' '))
rdd.take(7)

['La', 'programacion', 'y', 'la', 'ciencia', 'de', 'datos']

#### Aplicaciones:
- Inspeccionar una muestra inicial de datos.
- Depuración rápida de código al verificar los elementos de un RDD.

---

### 2. Función `max`

#### Descripción:
La función **`max`** se utiliza para encontrar el valor máximo en un RDD. Este método devuelve el elemento más grande según el orden natural de los datos o un criterio definido por el usuario.

#### Sintaxis:
```python
rdd.max()
```

#### Ejemplo de uso:

In [31]:
# Crear un RDD con números del 1 al 10
rdd = sc.parallelize(range(1, 11))

# Encontrar el valor máximo en el RDD
maximo = rdd.max()

# Imprimir el resultado
print(f"El valor máximo es: {maximo}")

El valor máximo es: 10


#### Aplicaciones:
- Identificar el valor más alto en un conjunto de datos.
- Encontrar el límite superior en métricas o medidas.


---

### 3. Función `saveAsTextFile`

#### Descripción:

La función **`saveAsTextFile`** se utiliza para guardar el contenido de un RDD en un archivo de texto. Cada elemento del RDD se escribe como una línea independiente en el archivo de salida. Esta función es útil para exportar los datos procesados a sistemas de archivos distribuidos como HDFS o al sistema local.

#### Sintaxis:

```python
rdd.saveAsTextFile(path)
```

- **`path`**: La ruta donde se guardarán los datos.

#### Ejemplo de uso:

In [32]:
# Crear un RDD con una lista de cadenas
rdd = sc.parallelize(["PySpark", "es", "muy", "útil"])

# Guardar el RDD en un archivo de texto
rdd.saveAsTextFile("salida_texto")

print("Datos guardados en la carpeta 'salida_texto'")

Datos guardados en la carpeta 'salida_texto'


> **Nota:** El archivo de salida será una carpeta que contendrá múltiples archivos de texto (partes) correspondientes a las particiones del RDD.

#### Aplicaciones:

- Exportar resultados procesados para su uso posterior.
- Guardar datos en formato legible para su análisis externo.
- Integrar datos procesados en PySpark con otros sistemas o herramientas.

In [33]:
# Crear un RDD con una lista de cadenas
rdd = sc.parallelize(["PySpark", "es", "muy", "útil"])

# Reducir el RDD a una sola partición
rdd_unica_particion = rdd.coalesce(1)

# Guardar el RDD en un archivo de texto con una sola partición
rdd_unica_particion.saveAsTextFile("salida_unica_particion")

print("Datos guardados en la carpeta 'salida_unica_particion' en un solo archivo.")

Datos guardados en la carpeta 'salida_unica_particion' en un solo archivo.
